# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [22]:
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [23]:
#replicate the data friles into data frame to prevent mistaken overwriting
master_df = school_data_complete
school_df = school_data
student_df = student_data
# calculating the number of unique schools
unique_schools = master_df ["school_name"].nunique()
# calculating the number of unique schools
student_count = student_data["student_name"].count()
# calculating the budget from the school_data file to avoid summming duplicates
district_budget = school_data['budget'].sum()
# calculating the average of each subject
average_math = master_df["math_score"].mean()
average_read = master_df["reading_score"].mean()

math_count = master_df['math_score'].count()
read_count = master_df['reading_score'].count()
num_math_pass = master_df.loc[master_df['math_score'] >= 70]['math_score'].count()
perc_math = num_math_pass/math_count
num_read_pass = master_df.loc[master_df['reading_score'] >= 70]['reading_score'].count()
perc_read = num_read_pass/read_count

# calculating the overall pass
overall_pass = ((perc_math+perc_read)/2)

districtstats = pd.DataFrame({"Unique Schools":[unique_schools],
                              "Student Count":[student_count],
                              "Total Budget":[district_budget],
                              "Average Math Score":[average_math],
                              "Average Reading Score":[average_read],
                              "% Passed Math":[perc_math],
                              "% Passed Reading":[perc_read],
                              "% Passed Overall":[overall_pass],})
districtstats

districtstats.style.format({"Student Count": "{:,.0f}", 
                            "Total Budget": "${:,.2f}",
                       "Average Math Score": "{:.1f}", 
                      "Average Reading Score": "{:.1f}", 
                       "% Passed Math": "{:.1%}", 
                       "% Passed Reading": "{:.1%}", 
                        "% Passed Overall": "{:.1%}"})

,Unique Schools,Student Count,Total Budget,Average Math Score,Average Reading Score,% Passed Math,% Passed Reading,% Passed Overall
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0%,85.8%,80.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [24]:
#creates summary table by school
summary_school = master_df.set_index("school_name").groupby(["school_name"])

# collects school types with name
types_school = school_df.set_index("school_name")["type"]

# takes total student count by school
studentcount_school = summary_school["Student ID"].count()

# calculates budget per school
school_budget = school_df.set_index("school_name")["budget"]

#calculates budget per student per school
student_budget = school_df.set_index("school_name")["budget"]/school_df.set_index("school_name")["size"]

#avg scores by school
math_average = summary_school["math_score"].mean()
read_average = summary_school["reading_score"].mean()

# % passing scores
math_passing = master_df[master_df["math_score"] >= 70].groupby("school_name")["Student ID"].count()/studentcount_school 
read_passing = master_df[master_df["reading_score"] >= 70].groupby("school_name")["Student ID"].count()/studentcount_school
overall_passing = (math_passing+read_passing)/2
#Jason what about my average calculation below? 
    #overall_passing = master_df[(master_df["reading_score"] >= 70) & (master_df["math_score"] >= 70)].groupby("school_name")["Student ID"].count()/studentcount_school

school_summary = pd.DataFrame({
                    "School Type": types_school,
                    "Total Students": studentcount_school,
                    "Total School Budget": school_budget,
                    "Per Student Budget": student_budget,
                    "Average Math Score": math_average,
                    "Average Reading Score": read_average,
                    '% Passing Math': math_passing,
                    '% Passing Reading': read_passing,
                    "% Overall Passing Rate": overall_passing
                    })

school_summary

#formatting
school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "% Overall Passing Rate": "{:.1%}"})





,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,74.3%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,95.6%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,73.4%
Ford High School,District,"2,739","$1,763,916",$644,77.1,80.7,68.3%,79.3%,73.8%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,95.3%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,73.8%
Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,94.4%
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,73.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,73.6%
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,95.3%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [25]:
#creating new table for top performers
top_performers = school_summary.sort_values("% Overall Passing Rate", ascending = False)
#have to reformat the new table, regardless of same values bein used and formatted before
top_performers.head().style.format({'Total Students': '{:,}',
                                   "Total School Budget": "${:,}", 
                                   "Per Student Budget": "${:.0f}", 
                                   "% Passing Math": "{:.1%}", 
                                   "% Passing Reading": "{:.1%}", 
                                   "% Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.0619,83.9758,94.1%,97.0%,95.6%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4183,83.8489,93.3%,97.3%,95.3%
Pena High School,Charter,962,"$585,858",$609,83.8399,84.0447,94.6%,95.9%,95.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.3515,83.8168,93.4%,97.1%,95.3%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.2742,83.9895,93.9%,96.5%,95.2%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [26]:

#use the tail of the top performers table sort and re-sort from worst to best
under_performers = top_performers.tail()
under_performers = under_performers.sort_values('% Overall Passing Rate')
under_performers.style.format({'Total Students': '{:,}', 
                               "Total School Budget": "${:,}", 
                               "Per Student Budget": "${:.0f}", 
                               "% Passing Math": "{:.1%}", 
                               "% Passing Reading": "{:.1%}", 
                               "% Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8427,80.7447,66.4%,80.2%,73.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7118,81.158,66.0%,80.7%,73.4%
Huang High School,District,"2,917","$1,910,635",$655,76.6294,81.1827,65.7%,81.3%,73.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.0725,80.9664,66.1%,81.2%,73.6%
Ford High School,District,"2,739","$1,763,916",$644,77.1026,80.7463,68.3%,79.3%,73.8%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [27]:
#caclulates the grade level average math scores for each school using groupby
math_9th = student_df.loc[student_df["grade"] == "9th"].groupby("school_name")["math_score"].mean()
math_10th = student_df.loc[student_df["grade"] == "10th"].groupby("school_name")["math_score"].mean()
math_11th = student_df.loc[student_df["grade"] == "11th"].groupby("school_name")["math_score"].mean()
math_12th = student_df.loc[student_df["grade"] == "12th"].groupby("school_name")["math_score"].mean()

#creates a new dataframe for just the grade data
math_scores = pd.DataFrame({
        "9th Grade": math_9th,
        "10th Grade": math_10th,
        "11th Grade": math_11th,
        "12th Grade": math_12th
})

math_scores 

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [28]:
#same as above
read_9th = student_df.loc[student_df["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
read_10th = student_df.loc[student_df["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
read_11th = student_df.loc[student_df["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
read_12th = student_df.loc[student_df["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

#same as above
reading_scores = pd.DataFrame({
        "9th Grade": read_9th,
        "10th Grade": read_10th,
        "11th Grade": read_11th,
        "12th Grade": read_12th
})

 
reading_scores 

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [29]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
spend_groups = ["<$585", "$585-615", "$615-645", "$645-675"]

In [33]:
master_df["spending_bins"] = pd.cut(master_df["budget"]/master_df["size"], spending_bins, labels = spend_groups)

#create summary table for spendng
spending_summary = master_df.groupby("spending_bins")

# same old calculations
avg_math = spending_summary['math_score'].mean()
avg_read = spending_summary['reading_score'].mean()
pass_math = master_df[master_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending_summary['Student ID'].count()
pass_read = master_df[master_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending_summary['Student ID'].count()
pass_overall = (pass_read+pass_math)/2

# new DF          
scores_by_spend = pd.DataFrame({
        "Average Math Score": avg_math,
        "Average Reading Score": avg_read,
        '% Passing Math': pass_math,
        '% Passing Reading': pass_read,
        "Overall Passing Rate": pass_overall           
    })
        

scores_by_spend.index.name = "Budget per Student"

#Style/format
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Budget per Student,,,,,
<$585,83.4,84.0,93.7%,96.7%,95.2%
$585-615,83.5,83.8,94.1%,95.9%,95.0%
$615-645,78.1,81.4,71.4%,83.6%,77.5%
$645-675,77.0,81.0,66.2%,81.1%,73.7%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [36]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [37]:

master_df['size_bins'] = pd.cut(master_df['size'], size_bins, labels = group_names)
#spending summary table for size
spending_summary2 = master_df.groupby("size_bins")

#same old calculations
avg_math = spending_summary2['math_score'].mean()
avg_read = spending_summary2['reading_score'].mean()
pass_math = master_df[master_df['math_score'] >= 70].groupby("size_bins")['Student ID'].count()/spending_summary2['Student ID'].count()
pass_read = master_df[master_df['reading_score'] >= 70].groupby("size_bins")['Student ID'].count()/spending_summary2['Student ID'].count()
pass_overall = (pass_read+pass_math)/2
         
#new DF          
scores_by_size = pd.DataFrame({
        "Average Math Score": avg_math,
        "Average Reading Score": avg_read,
        '% Passing Math': pass_math,
        '% Passing Reading': pass_read,
        "Overall Passing Rate": pass_overall           
    })

scores_by_size.index.name = "School Size"
scores_by_size = scores_by_size.reindex(group_names)

#style
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.8,84.0,94.0%,96.0%,95.0%
Medium (1000-2000),83.4,83.9,93.6%,96.8%,95.2%
Large (2000-5000),77.5,81.2,68.7%,82.1%,75.4%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [46]:

spend_by_type = master_df.groupby("type")

#same old calculations 
avg_math = spend_by_type['math_score'].mean()
avg_read = spend_by_type['math_score'].mean()
pass_math = master_df[master_df['math_score'] >= 70].groupby('type')['Student ID'].count()/spend_by_type['Student ID'].count()
pass_read = master_df[master_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/spend_by_type['Student ID'].count()
pass_overall = (pass_read+pass_math)/2
# new DF           
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "% Overall Passing Rate": pass_overall})

scores_by_type.index.name = "Type of School"


#style
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              '% Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,95.2%
District,77.0,77.0,66.5%,80.9%,73.7%
